# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [82]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error, make_scorer

from sklearn.model_selection import cross_val_score
import scipy.stats as st
import time


In [62]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [65]:
spaceship["Cabin"] = spaceship["Cabin"].str.split('/').str[0]
spaceship.drop(["PassengerId", "Name"], axis =1, inplace = True)
spaceship = spaceship.dropna()
spaceship.isnull().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [67]:
categorical_data = spaceship.select_dtypes("object")
categorical_data

# get dummies for categorical data
data_dummies = pd.get_dummies(categorical_data, drop_first=True)
data_dummies = data_dummies.astype(int)

# numerical features
numerical_data = spaceship.select_dtypes("number")

features = pd.concat([numerical_data, data_dummies], axis=1) # combine numerical and categorical features
target = spaceship["Transported"].astype(int) # define the target

In [69]:
# performing train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

# normalize numerical features
normalizer = MinMaxScaler()
normalizer.fit(X_train)

#scale numerical features
scaler = StandardScaler()
scaler.fit(X_train)


X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

X_train_norm_df = pd.DataFrame(X_train_norm, columns = X_train.columns, index=X_train.index)
X_test_norm_df = pd.DataFrame(X_test_norm, columns = X_test.columns, index=X_test.index)

X_train_standarized_np = scaler.transform(X_train)
X_test_standarized_np = scaler.transform(X_test)

X_train_standarized_df = pd.DataFrame(X_train_standarized_np, columns = X_train.columns, index=X_train.index)
X_test_standarized_df  = pd.DataFrame(X_test_standarized_np, columns = X_test.columns, index=X_test.index)

X_train_standarized_df.head(5)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
6201,0.633057,-0.353053,-0.168253,-0.316237,0.903105,3.331720,1.717450,-0.510317,-0.731759,-0.318377,-0.313423,-0.244405,2.931548,-0.696645,-0.652264,-0.019229,-0.325717,-1.507794,-0.158129
6106,0.633057,-0.353053,-0.290009,-0.316237,-0.280524,-0.271427,-0.582259,-0.510317,1.366570,-0.318377,-0.313423,-0.244405,-0.341117,-0.696645,1.533122,-0.019229,3.070150,-1.507794,-0.158129
7095,0.702161,-0.353053,-0.290009,-0.316237,-0.280524,-0.271427,-0.582259,1.959564,1.366570,-0.318377,-0.313423,-0.244405,-0.341117,1.435451,-0.652264,-0.019229,-0.325717,-1.507794,-0.158129
3060,-1.992884,-0.353053,-0.290009,-0.316237,-0.280524,-0.271427,-0.582259,-0.510317,-0.731759,-0.318377,-0.313423,-0.244405,-0.341117,-0.696645,1.533122,-0.019229,-0.325717,-1.507794,-0.158129
4101,0.909472,1.106502,-0.290009,-0.314455,-0.280524,-0.271427,-0.582259,1.959564,-0.731759,-0.318377,-0.313423,-0.244405,2.931548,-0.696645,-0.652264,-0.019229,-0.325717,0.663221,-0.158129


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [72]:
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=20), n_estimators=100, max_samples=1000)

bagging.fit(X_train_standarized_df, y_train)
bagging.score(X_test_standarized_df, y_test)

0.8078344419807835

- Evaluate your model

In [75]:
pred = bagging.predict(X_test_standarized_df)

print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"MSE, {mean_squared_error(pred, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(pred, y_test): .2f}")
print(f"R2 score, {bagging.score(X_test_standarized_df, y_test): .2f}")

MAE,  0.19
MSE,  0.19
RMSE,  0.44
R2 score,  0.81


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

- Run Grid Search

In [84]:
#define the parameter
parameter_grid = {"max_depth": [10, 50],
                  "min_samples_split": [4, 16],
                  "max_leaf_nodes": [250, 100],
                  "max_features": ["sqrt", "log2"]} # In example we're going to test 2 * 2 * 2 * 2 = 16 combinations of hyperparameters


# We create an instance or our machine learning model
dt = DecisionTreeClassifier(random_state=123)

# We need to set this two variables to be able to compute a confidence interval
confidence_level = 0.95
folds = 10

# Now we need to create an intance of the GridSearchCV class
gs = GridSearchCV(dt, param_grid=parameter_grid, cv=folds, verbose=10) # Here the "cv" allows you to define the number of folds to use.

start_time = time.time()
gs.fit(X_train_standarized_df, y_train)
end_time = time.time()

print("\n")
print(f"Time taken to find the best combination of hyperparameters among the given ones: {end_time - start_time: .4f} seconds")
print("\n")


print(f"The best combination of hyperparameters has been: {gs.best_params_}")
print(f"The R2 is: {gs.best_score_: .4f}")

results_gs_df = pd.DataFrame(gs.cv_results_).sort_values(by="mean_test_score", ascending=False)

#print(results_df.head())
gs_mean_score = results_gs_df.iloc[0,-3]
gs_sem = results_gs_df.iloc[0,-2] / np.sqrt(10)

gs_tc = st.t.ppf(1-((1-confidence_level)/2), df=folds-1)
gs_lower_bound = gs_mean_score - ( gs_tc * gs_sem )
gs_upper_bound = gs_mean_score + ( gs_tc * gs_sem )

print(f"The R2 confidence interval for the best combination of hyperparameters is: \
    ({gs_lower_bound: .4f}, {gs_mean_score: .4f}, {gs_upper_bound: .4f}) ")

#display(results_df)

# Let's store the best model
best_model = gs.best_estimator_

# Now is time evaluate the model in the test set
y_pred_test = best_model.predict(X_test_standarized_df)
y_pred_test = best_model.predict(X_test_standarized_df)

y_pred_test = best_model.predict(X_test_standarized_df)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 1/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.782 total time=   0.0s
[CV 2/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 2/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.765 total time=   0.0s
[CV 3/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 3/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.782 total time=   0.0s
[CV 4/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 4/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.773 total time=   0.0s
[CV 5/10; 1/16] START max_depth=10, max_features=sqrt

- Evaluate your model

In [86]:
print("\n")
print(f"Test MAE: {mean_absolute_error(y_pred_test, y_test): .4f}")
print(f"Test MSE: {mean_squared_error(y_pred_test, y_test): .4f}")
print(f"Test RMSE: {root_mean_squared_error(y_pred_test, y_test): .4f}")
print(f"Test R2 score:  {best_model.score(X_test_standarized_df, y_test): .4f}")
print("\n")



Test MAE:  0.2380
Test MSE:  0.2380
Test RMSE:  0.4878
Test R2 score:   0.7620


